In [74]:
import numpy as np
import pandas as pd
import json
import datetime
import spacy
from tqdm import tqdm
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import plotly.express as px
from coins import get_top_coins


In [42]:
f = open('./Telegram_Data/result.json', encoding='utf8')
data = json.load(f)
for key in data.keys():
    print(key)

  0%|          | 37/10000 [00:16<12:36, 13.16it/s]

name
type
id
messages


In [55]:
df = pd.DataFrame(data['messages'])
df = df.iloc[0:1000]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1000 non-null   int64  
 1   type                 1000 non-null   object 
 2   date                 1000 non-null   object 
 3   from                 929 non-null    object 
 4   from_id              998 non-null    object 
 5   reply_to_message_id  494 non-null    float64
 6   text                 1000 non-null   object 
 7   edited               23 non-null     object 
 8   file                 1 non-null      object 
 9   thumbnail            1 non-null      object 
 10  media_type           1 non-null      object 
 11  sticker_emoji        1 non-null      object 
 12  width                2 non-null      float64
 13  height               2 non-null      float64
 14  forwarded_from       3 non-null      object 
 15  actor                1 non-null      ob

,id,type,date,from,from_id,reply_to_message_id,text,edited,file,thumbnail,...,actor_id,action,members,photo,message_id,contact_information,contact_vcard,mime_type,duration_seconds,via_bot
0,1903163,message,2021-05-01T00:00:14,Seflipe,user1697045867,1903101.0,Double check the address,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1903164,message,2021-05-01T00:01:33,Seflipe,user1697045867,1903138.0,To get help to understand the product not how ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1903168,message,2021-05-01T00:03:13,Seflipe,user1697045867,1903122.0,You can find good youtube channels.. Check the...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1903188,message,2021-05-01T00:13:37,Neto | Crypto.com,user1395411322,1903160.0,"Are you doing withdrawals to another user, DeF...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1903191,message,2021-05-01T00:14:10,PauLaLa,user1501886330,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")

if not "language_detector" in Language.factories:
    Language.factory("language_detector", func=get_lang_detector)

nlp.add_pipe('language_detector', last=True)

In [57]:
def extract_features(x):
    doc = nlp(x)
    lang_dict = doc._.language
    language = lang_dict['language']
    entities = [ent.text for ent in doc.ents]
    return language, entities

df['language'], df['entities'] = zip(*[extract_features(str(x)) for x in tqdm(df['text'])])

In [60]:
df[['text', 'language','entities']]

,text,language,entities
0,Double check the address,en,[]
1,To get help to understand the product not how ...,en,[]
2,You can find good youtube channels.. Check the...,en,[]
3,"Are you doing withdrawals to another user, DeF...",en,"[DeFi Wallet, External]"
4,Yes,tr,[]
...,...,...,...
995,Do your own research Dheer,en,[Dheer]
996,[Make sure you've accepted the awards in the A...,en,[App]
997,prepare ammunition to buy?,en,[]
998,"Nope , i even not opened my account , or even ...",en,[]


In [61]:
df.language.unique()

array(['en', 'tr', 'it', 'es', 'UNKNOWN', 'nl', 'fi', 'ro', 'sv', 'hu',
       'ca', 'tl', 'da', 'fr', 'cy', 'no', 'so', 'vi', 'af', 'de', 'sk',
       'sw', 'pl', 'id', 'pt', 'hr', 'et', 'sq', 'sl'], dtype=object)

In [62]:
df_eng = df[df.language.values == 'en']
df_new = df_eng.filter(items = ['id','text', 'date'])
# df_new = df_new[df_new['text'].str.lower().str.contains('bitcoin|doge|btc') & (df_new['text'].str.len() > 1)]

In [75]:
top_coins = get_top_coins(20)

related_counts = {}

for coin in top_coins:
    name_matches = df_new['text'].str.lower().str.contains(coin['name'].lower()).sum()
    symbol_matches = df_new['text'].str.lower().str.contains(coin['symbol'].lower()).sum()
    total_matches = name_matches + symbol_matches
    related_counts[coin['name']] = total_matches

top_5_coins = dict(sorted(related_counts.items(), key=lambda item: item[1], reverse=True)[:5])

top_5_df = pd.DataFrame(top_5_coins.items(), columns=['Coin Name', 'Related Text Count'])

print(top_5_df)

    Coin Name  Related Text Count
0     Bitcoin                  10
1    Dogecoin                  10
2    Ethereum                   1
3  Tether USD                   0
4         BNB                   0


In [63]:
df_new.head()

,id,text,date
36,1903358,Bitcoin,2021-05-01T01:04:38
118,1903552,"Thank you! I was just wondering, after factori...",2021-05-01T01:54:32
119,1903555,"The 0.0004 BTC withdrawal fee is flat, and the...",2021-05-01T01:55:37
123,1903565,For example let’s say I have $1000sgd worth of...,2021-05-01T02:02:14
125,1903568,Is it the same amount of BTC still?\n\nMinus t...,2021-05-01T02:03:15
134,1903582,"Though, as long as the BTC amount is still cor...",2021-05-01T02:07:02
267,1904094,Doge is going craY,2021-05-01T03:56:05
278,1904111,"Anyway, is doge a good crypto for long term in...",2021-05-01T04:00:32
289,1904178,"Ok bro, any reason for that? Bases on your exp...",2021-05-01T04:10:40
332,1904369,Who else going to mars With doge😁,2021-05-01T04:46:27


In [64]:
sid_obj = SentimentIntensityAnalyzer()

In [65]:
def Vader_senti(x):
    scores = sid_obj.polarity_scores(x)
    return scores['neg'],scores['neu'],scores['pos'],scores['compound']

df_new[['vader_neg','vader_neu','vader_pos','vader_compound']] = [Vader_senti(x) for x in tqdm(df_new['text'])]




100%|██████████| 20/20 [00:00<00:00, 4197.24it/s]


In [66]:
df_new.head()

,id,text,date,vader_neg,vader_neu,vader_pos,vader_compound
36,1903358,Bitcoin,2021-05-01T01:04:38,0.000,1.000,0.000,0.0000
118,1903552,"Thank you! I was just wondering, after factori...",2021-05-01T01:54:32,0.044,0.791,0.165,0.7312
119,1903555,"The 0.0004 BTC withdrawal fee is flat, and the...",2021-05-01T01:55:37,0.073,0.891,0.036,-0.2732
123,1903565,For example let’s say I have $1000sgd worth of...,2021-05-01T02:02:14,0.091,0.741,0.169,-0.0258
125,1903568,Is it the same amount of BTC still?\n\nMinus t...,2021-05-01T02:03:15,0.000,1.000,0.000,0.0000


In [67]:
def parse_date(x):
    """
    Function to extract date from the initial dataset
    """
    date_time_obj = datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')
    return date_time_obj.date()

df_new['Day'] = [parse_date(x) for x in tqdm(df_new['date'])]




100%|██████████| 20/20 [00:00<00:00, 2576.12it/s]


In [68]:
# Filter rows containing 'bitcoin'
df_btc = df_new[df_new['text'].str.lower().str.contains('btc|bitcoin')]

# Filter rows containing 'doge'
df_doge = df_new[df_new['text'].str.lower().str.contains('doge')]
avg_sent = df_doge.groupby(['Day']).agg({'vader_compound' : ['mean', 'count']})
avg_sent.columns = ['_'.join(str(i) for i in col) for col in avg_sent.columns]
avg_sent.reset_index(inplace=True)
avg_sent = avg_sent.rename(columns={'vader_compound_mean': 'doge_mean', 'vader_compound_count': 'doge_count'})
print(avg_sent)

print("-" * 50)
avg_sent2 = df_btc.groupby(['Day']).agg({'vader_compound' : ['mean', 'count']})
avg_sent2.columns = ['_'.join(str(i) for i in col) for col in avg_sent2.columns]
avg_sent2.reset_index(inplace=True)
avg_sent2 = avg_sent2.rename(columns={'vader_compound_mean': 'btc_mean', 'vader_compound_count': 'btc_count'})
print(avg_sent2)

          Day  doge_mean  doge_count
0  2021-05-01    0.11685          10
--------------------------------------------------
          Day  btc_mean  btc_count
0  2021-05-01   0.09411         10


In [69]:
avg_sent['Sentiment'] = ["Negative" if x<0 else "Positive"
                         for x in avg_sent['doge_mean']]
avg_sent2['Sentiment'] = ["Negative" if x<0 else "Positive"
                         for x in avg_sent2['btc_mean']]

In [70]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)

fig.add_trace(px.histogram(avg_sent, x="Day", y="doge_count", color="Sentiment", nbins=20).data[0], row=1, col=1)

fig.add_trace(px.histogram(avg_sent2, x="Day", y="btc_count", color="Sentiment", nbins=20).data[0], row=1, col=2)

fig.update_layout(
    title_text='Vader Sentiment Analysis Results',
    bargap=0.2, 
    bargroupgap=0.1 
)

fig.update_xaxes(title_text="Doge Coin", row=1, col=1)
fig.update_xaxes(title_text="Bitcoin", row=1, col=2)

fig.show()